# Notebook: Processamento de Vôos
Este notebook processa ficheiros Excel exportados do flights4staff.ana.pt.
- Processa chegadas e partidas separadamente
- Filtra, ordena e formata os dados
- Gera ficheiros Excel finais prontos para abrir/imprimir


In [ ]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Alignment
from google.colab import files

In [ ]:
# ===========================
# FUNÇÃO AUXILIAR: FORMATA EXCEL
# ===========================
def format_excel(df, output_file_name, sum_column_name=None):
    """
    Formata um DataFrame e grava em Excel:
    - Ajusta largura de colunas
    - Centraliza o texto
    - Aplica cores alternadas nas linhas
    - Adiciona soma de coluna no final (opcional)
    """
    df.to_excel(output_file_name, index=False)
    wb = openpyxl.load_workbook(output_file_name)
    sheet = wb.active

    # Ajusta largura das colunas
    for column in sheet.columns:
        max_length = 0
        column_letter = openpyxl.utils.get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        sheet.column_dimensions[column_letter].width = max_length + 1

    # Centraliza texto
    for row in sheet.iter_rows(min_row=2, values_only=False):
        for cell in row:
            cell.alignment = Alignment(horizontal='center', vertical='center')

    # Aplica cores alternadas
    fill = PatternFill(start_color='d9d9d9', end_color='d9d9d9', fill_type='solid')
    for i, row in enumerate(sheet.iter_rows(min_row=2), start=2):
        if i % 2 == 0:
            for cell in row:
                cell.fill = fill

    # Adiciona soma da coluna (opcional)
    if sum_column_name and sum_column_name in df.columns:
        total_value = df[sum_column_name].sum()
        max_row = len(df) + 2
        col_idx = df.columns.get_loc(sum_column_name) + 1
        sheet.cell(row=max_row, column=col_idx, value=f'Total {sum_column_name}: {total_value}')

    wb.save(output_file_name)

## Processa Chegadas

In [ ]:
print('###### ------------------------------------------ ########')
print('###### CARREGAR EM BAIXO E ESCOLHER VÔOS CHEGADAS ########')
print('###### ------------------------------------------ ########')
uploaded = files.upload()
excel_file_name = next(iter(uploaded))

# Carrega Excel
df = pd.read_excel(excel_file_name)

# Filtra colunas
desired_columns = ['Mov Type', 'Schedule Date Time', 'Flight Origin', 'Flight Destination', 'Flight Nr', 'Capacity', 'Classification']
df = df[desired_columns]

# Filtra Classification
df = df[df['Classification'].isin(['I', 'NS'])]

# Filtra capacidade mínima >=20
df = df[df['Capacity'] >= 20]

# Ordena por horário
df.sort_values(by='Schedule Date Time', inplace=True)

# Coluna numerada
df.insert(0, 'Quant', range(1, 1 + len(df)))

# Renomeia colunas
df.columns = ['___', 'Tipo', 'Hora', 'Origem', 'APIS', 'Vôo', 'Cap.', 'Class.']

output_file_name = 'Arrivals_modif.xlsx'
format_excel(df, output_file_name, sum_column_name='Cap.')
files.download(output_file_name)
print(f"Chegadas filtradas e formatadas salvas em '{output_file_name}'")

## Processa Partidas

In [ ]:
print('###### ---------------------------------------------------------- ########')
print('###### AGORA ESCOLHER FICHEIRO COM P A R T I D A S ########')
print('###### ---------------------------------------------------------- ########')
uploaded = files.upload()
excel_file_name = next(iter(uploaded))

# Carrega Excel
df = pd.read_excel(excel_file_name)
df = df[desired_columns]
df = df[df['Classification'].isin(['I', 'NS'])]
df = df[df['Capacity'] >= 20]
df.sort_values(by='Schedule Date Time', inplace=True)
df.insert(0, 'Quant', range(1, 1 + len(df)))
df.columns = ['___', 'Tipo', 'Hora', 'Origem', 'Destino', 'Vôo', 'Cap.', 'Class.']

output_file_name = 'Departures_modif.xlsx'
format_excel(df, output_file_name, sum_column_name='Cap.')
files.download(output_file_name)
print(f"Partidas filtradas e formatadas salvas em '{output_file_name}'")